In [1]:
import sqlite3
import pandas as pd
import requests
import json
import auxiliares4 as aux

In [2]:
conn = sqlite3.connect('data_bb.db')

conn

In [3]:
query = '''
    SELECT TICKER, COUNT(VALOR) AS N_FECHAS
    FROM PRECIOS
    GROUP BY TICKER
    ORDER BY N_FECHAS
'''

df = pd.read_sql(query, conn)
df.head()

,ticker,N_FECHAS
0,CFR CC Equity,1035
1,CENCOSHO CC Equity,1077
2,MALLPLAZ CC Equity,1304
3,PILMAIQ CC Equity,1568
4,SMU CC Equity,1678


In [4]:
user='1090208'
pwd='cyTG5VWgjynj'

serie = 'F073.TCO.PRE.Z.D'

url = f"https://si3.bcentral.cl/SieteRestWS/SieteRestWS.ashx?" + \
    f"user={user}&pass={pwd}&lastdate=2023-09-29&" + \
    f"timeseries={serie}&function=GetSeries"

response = requests.get(url)
data = response.json()["Series"]["Obs"]

df1 = pd.DataFrame(data)
df1

,indexDateString,value,statusCode
0,09-08-1982,55.65,OK
1,10-08-1982,62.79,OK
2,11-08-1982,64.07,OK
3,12-08-1982,60.52,OK
4,13-08-1982,56.87,OK
...,...,...,...
15022,25-09-2023,889.21,OK
15023,26-09-2023,900.14,OK
15024,27-09-2023,904.31,OK
15025,28-09-2023,904.62,OK


In [5]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)
tables

,name
0,PRECIOS
1,TICKERNEMO


In [6]:
query_nemotecnico = "SELECT * FROM TICKERNEMO WHERE nombre LIKE '%CENCOSUD%';"
nemotecnico_cencosud = pd.read_sql_query(query_nemotecnico, conn)
nemotecnico_cencosud

,index,ticker,nombre,nemo
0,10,CENCOSUD CC Equity,Cencosud SA,CENCOSUD


In [7]:
tickers_con_datos = ['CENCOSUD']  # Tickers con datos según la muestra
fecha_prueba_con_datos = "2023-09-29"  # Fecha dentro del rango de la muestra de datos
df_rentabilidades_con_datos = aux.calcular_rentabilidades(conn, fecha_prueba_con_datos, tickers_con_datos)
df_rentabilidades_con_datos

,Instrumento,Ult. Precio,Rent 1d,Rent 1m,Rent ytd*,Rent 1y*,Rent 5y*
0,CENCOSUD,1685.0,0.60%,-7.42%,28.97%,37.55%,1.50%
1,IPSA,5833.36,0.15%,-3.23%,10.86%,15.85%,2.00%
2,Dólar,906.84,0.25%,5.74%,5.51%,-6.62%,6.51%
3,Fecha de referencia:,2023-09-29,,,,,


In [8]:
nombre_archivo = "req4.xlsx"
# Guarda el DataFrame en un archivo Excel
df_rentabilidades_con_datos.to_excel(nombre_archivo, index=False)
print(f"El DataFrame se ha guardado en {nombre_archivo}")

El DataFrame se ha guardado en req4.xlsx
